#### 1. Import pandas library

In [2]:
# imports
import pandas as pd
import os
import this

# listing the files in the directory to id the users table
os.listdir()

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


['main.ipynb',
 'posts_table.csv',
 'weather.ipynb',
 '.ipynb_checkpoints',
 'users_table.csv']

#### 2. Import users table:

In [3]:
file = "users_table.csv"
users = pd.read_csv(file)

#### 3. Rename Id column to userId

In [4]:
users.rename(columns = {"Id": "userId"}, inplace = True)
users

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40320,55743,1,2014-09-13 21:03:50,AussieMeg,2014-09-13 21:18:52,NaN,NaN,NaN,0,0,0,5026902,NaN,http://graph.facebook.com/665821703/picture?ty...
40321,55744,6,2014-09-13 21:39:30,Mia Maria,2014-09-13 21:39:30,NaN,NaN,NaN,1,0,0,5026998,NaN,NaN
40322,55745,101,2014-09-13 23:45:27,tronbabylove,2014-09-13 23:45:27,NaN,United States,NaN,0,0,0,481766,NaN,https://www.gravatar.com/avatar/faa7a3fdbd8308...
40323,55746,106,2014-09-14 00:29:41,GPP,2014-09-14 02:05:17,NaN,NaN,"<p>Stats noobie, product, marketing &amp; medi...",1,0,0,976289,NaN,https://www.gravatar.com/avatar/6d9e9fa6b783a3...


#### 4. Import posts table:

In [5]:
posts_file = 'posts_table.csv'
posts = pd.read_csv(posts_file)

#### 5. Rename Id column to postId and OwnerUserId to userId

In [6]:
posts.rename(columns = {"Id": "postId", "OwnerUserId" : "userId"}, inplace = True)
posts

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [7]:
new_users = users[["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]
new_users.head()

new_posts = posts[["postId", "Score", "userId", "ViewCount", "CommentCount"]]
new_posts.head()

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [8]:
merged = new_users.merge(new_posts, on = "userId")
merged.head()

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [9]:
def null_cols(ds):
    """check whether the value in each field is missing (null) and return either 
    True or False for each field, totaling up the number of True values by column."""
    return ds.isnull().sum()

def na_cols(ds):
    """Does the same as null_cols, but returns the value as a percentage. 
    Useful to decide where to drop."""
    return ds.isna().mean().round(4) * 100

merged_null_cols = null_cols(merged)
merged_na_cols = na_cols(merged)

In [10]:
# 'ViewCount' w/ 23.5K+, the rest have no null-values (but there are some with 0's)
merged_null_cols

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

In [11]:
# 'ViewCount' w/ 60.5% of null-values
merged_na_cols

userId           0.0
Reputation       0.0
Views            0.0
UpVotes          0.0
DownVotes        0.0
postId           0.0
Score            0.0
ViewCount       60.5
CommentCount     0.0
dtype: float64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [12]:
# Droping the columns with null values is (always?) a judgment call. 
# In a discussion on analyticsvidhya.com 
# (what-should-be-the-allowed-percentage-of-missing-values/2456), the sugestion is to drop
# after the 30% ceiling has been reached. Still, it will depend.

# For starters I'll check what entities have the missing values and what don't;
missing = merged.loc[(merged["ViewCount"].notna() != True)]

In [13]:
missing

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
...,...,...,...,...,...,...,...,...,...
38939,32036,7227,991,2664,143,28258,0,NaN,0
38942,35921,113,2,1,0,16397,-2,NaN,1
38944,37890,111,0,0,0,41240,1,NaN,3
38946,42201,131,0,0,0,10683,3,NaN,0


In [36]:
# 2.6K+ unique entities have null 'ViewCount'
unique_id = missing["userId"].unique()
len(unique_id)

2618

In [35]:
# from 8.1K entities, so about 30%.
merged["userId"].unique()
len()

8138

In [44]:
# This show how many by each userId
grouped = merged.groupby("userId")
grouped.groups

grouped.get_group(-1)
grouped.size()

userId
-1        123
 5        117
 6         12
 7          2
 8        121
         ... 
 45934      1
 46192      1
 46522      1
 52371      1
 55226      1
Length: 8138, dtype: int64

In [53]:
# W/more time I'd check for other relations between the NaN's, userId's, postId's, 
# CommentCount... For instance, I find it odd that 123 entitites w/NaN's in ViewCount
# are for userId -1 (which is an odd userId by itself), and that the only different
# among these seems to be the postId.. This seems more like a systematic error than
# anything else.

# Since I don't have the time at the moment, I'll just fill the NaN's as 0
merged["ViewCount"].fillna(0, inplace = True)
merged["ViewCount"].value_counts()

0.0       23572
150.0        43
98.0         43
122.0        42
156.0        41
          ...  
4414.0        1
9772.0        1
1796.0        1
4976.0        1
2174.0        1
Name: ViewCount, Length: 3403, dtype: int64

In [55]:
merged

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
...,...,...,...,...,...,...,...,...,...
38957,45934,11,1,0,0,34003,1,115.0,2
38958,46192,36,1,0,0,40667,5,326.0,2
38959,46522,235,13,27,1,17461,3,166.0,0
38960,52371,221,2,0,0,27237,24,3357.0,5


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [54]:
# Cgecking the datatypes
merged.dtypes

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [59]:
# Changing ViewCount from floats to ints
merged = merged.astype({'ViewCount': 'int64'})

In [61]:
merged.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object